# Procesamiento de Lenguaje Natural


**Pablo Martínez Olmos, Vanessa Gómez Verdejo, Emilio Parrado Hernández**

Departamento de Teoría de la Señal y Comunicaciones

**Universidad Carlos III de Madrid**

<img src='http://www.tsc.uc3m.es/~emipar/BBVA/INTRO/img/logo_uc3m_foot.jpg' width=400 />


In [6]:
%matplotlib inline  
# Figures plotted inside the notebook
%config InlineBackend.figure_format = 'svg'  
# High quality figures
import matplotlib.pyplot as plt
import numpy as np

# Introducción

Hasta ahora hemos estado trabajando con datos de tipo  numérico o categórico. En esta sesión vamos a ver cómo trabajar con nuestros datos cuando éstos son cadenas de texto. A diferencia de los datos categóricos, en los que tenemos cadenas de texto asociadas a diferentes categorías y que podemos codificar fácilmente (por ejemplo, con un one-hot-encoding), cuando hablamos aquí de información textual nos referimos a frases, documentos y/o corpus de datos con estructura mucho más compleja. Idealmente, a partir de estos datos textuales tenemos que extraer la información necesaria (a poder ser incluyendo contenido semántico) y vectorizarla adecuadamente para poder utilizar o usar modelos de aprendizaje a partir de ella.

En general, gran parte de la forma en que nos comunicamos hoy en día es a través de texto escrito, ya sea en servicios de mensajería, medios sociales y/o correo electrónico. Así, por ejemplo, en servicios/aplicaciones como TripAdvisor, Booking, Amazon, etc., los usuarios escriben reseñas de restaurantes/negocios, hoteles, productos para compartir sus opiniones sobre su experiencia. Estas reseñas, todas escritas en formato de texto, contienen una gran cantidad de información que sería útil responder preguntas relevantes para el negocio usando métodos de aprendizaje automático, por ejemplo, para predecir el mejor restaurante en una determinada zona. 

Este tipo de tarea (preprocesado) se denomina **procesamiento del lenguaje natural** (Natural Language Processing, NLP).
El NLP es un subcampo de la lingüística, la informática y la inteligencia artificial que se ocupa de las interacciones entre los ordenadores (o procesadores) y el lenguaje humano; en particular engloba un conjunto de técnicas para permitir que los ordenadores procesen y analicen grandes cantidades de texto.



# Pipeline para el procesado de texto 

Como sabemos, los algoritmos de ML procesan números, no palabras, por lo que necesitamos transformar el texto en números significativos que contengan la información relevante de los documentos. Este proceso de convesión de texto a números es lo que llamaremos **vectorización**. 

No obstante, para tener una representación útil, se requieren normalmente algunos pasos de **preprocesamiento** previo que limpien y homogenizen los documentos: tokenización, eliminación de *stop-words*, lematización, etc.
La siguiente figura muestra los diferentes pasos que debemos seguir para procesar nuestros documentos hasta poder ser utilizados por nuestro modelo de aprendizaje:

<img src="http://www.tsc.uc3m.es/~vanessa/Figs_notebooks/BBVA/NLP/PipelineNLP.png" width="80%"> 

A lo largo de este notebook, veremos las herramientas que tenemos disponibles en Python para llevar a cabo todos estos pasos. Concretamente, nos centraremos en el uso de dos librerias:
* [NLTK, Natural Language ToolKit](https://www.nltk.org/). Esta libreria es una excelente biblioteca de NLP escrita en Python por expertos tanto del mundo académico como de la industria. NLTK permite crear aplicaciones con datos textuales rápidamente, ya que proporciona un conjunto de clases básicas para trabajar con corpus de datos, incluyendo colecciones de textos (corpus), listas de palabras clave, clases para representar y operar con datos de tipo texto (documentos, frases, palabras, ...) y funciones para realizar tareas comunes de NLP (conversión a token, conteo de palabras, ...). Por lo que va a ser de gran ayuda para el preprocesado de los documentos.


* [SpaCy](https://spacy.io/) es una biblioteca de código abierto para procesamiento avanzado del lenguaje natural en Python. SpaCy está diseñado específicamente para su uso en producción. A diferencia de NLTK, SpaCy tiene una estructura orientada a objetos. Por ejemplo, al tokenizar texto, cada token es un objeto con atributos y propiedades específicas. SpaCy admite más de 64 idiomas, incluyendo modelos estadísticos ya entrenados para [17 de ellos](https://spacy.io/usage/models) (incluyendo modelos basados en [transformers](https://spacy.io/usage/v3), la última revolución en NLP).


* [Gensim](https://pypi.org/project/gensim/) es otra librería de Python para el modelado por temáticas (*topic modeling*), la indexación de documentos y tareas de recuperación de la información para documentos. Está diseñada para operar con grandes cantidades de información (con implementaciones eficientes y paralelizables/distribuidas) y nos va a ser de gran ayuda para la vectorización de nuestros corpus de datos una vez preprocesados.

# 1. Base de datos 

NLTK incluye diferentes corpus de datos para probar nuestras herramientas. Podemos encontrar información sobre todos ellos en [corpus NLTK] (https://www.nltk.org/book/ch02.html).

A pesar de que podemos usar NLTK para realizar el preprocesamiento de texto, usaremos [spaCy](https://spacy.io/) para tal fin.


In [7]:
import nltk

## El objeto CorpusReader
Para empezar a trabajar vamos a utilizar el corpus **movie_reviews**, uno de los corpus de datos incluidos en NLTK y formado por 2000 documentos de texto con reseñas de diferentes películas donde además se indica si estas reseñas son positivas o negativas.

La siguiente celda de código muestra cómo cargar el corpus...


In [8]:
from nltk.corpus import movie_reviews
nltk.download('movie_reviews')
movie_reviews

[nltk_data] Downloading package movie_reviews to
[nltk_data]     /Users/olmos/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


<CategorizedPlaintextCorpusReader in '/Users/olmos/nltk_data/corpora/movie_reviews'>

Al cargar el corpus, se genera un objeto de tipo `CorpusReader`, denominado `movie_reviews`, con el contenido del corpus. Dado que un corpus es una colección de documentos/textos, podemos ver qué documentos componen este corpus usando la función `.fileids()`

In [9]:
len(movie_reviews.fileids())

2000

In [10]:
movie_reviews.fileids()[:10]

['neg/cv000_29416.txt',
 'neg/cv001_19502.txt',
 'neg/cv002_17424.txt',
 'neg/cv003_12683.txt',
 'neg/cv004_12641.txt',
 'neg/cv005_29357.txt',
 'neg/cv006_17022.txt',
 'neg/cv007_4992.txt',
 'neg/cv008_29326.txt',
 'neg/cv009_29417.txt']

También podemos ver las categorías de este problema con el atributo `.categories()`.

In [11]:
movie_reviews.categories()

['neg', 'pos']

In [12]:
movie_reviews.categories('neg/cv000_29416.txt')

['neg']

Si queremos, podemos acceder a un **documento** específico en el corpus y extraer su contenido sin procesar con la función `.raw()`. 

In [13]:
raw_text = movie_reviews.raw('neg/cv007_4992.txt')
print(raw_text)

that's exactly how long the movie felt to me . 
there weren't even nine laughs in nine months . 
it's a terrible mess of a movie starring a terrible mess of a man , mr . hugh grant , a huge dork . 
it's not the whole oral-sex/prostitution thing ( referring to grant , not me ) that bugs me , it's the fact that grant is annoying . 
not just adam sandler-annoying , we're talking jim carrey-annoying . 
since when do eye flutters and nervous smiles pass for acting ? 
but , on the other hand , since when do really bad slapstick ( a fistfight in the delivery room culminating in grant's head in joan cusack's lap--a scene he paid $60 to have included in the movie ) and obscene double entendres ( robin williams , the obstetrician , tells grant's pregnant girlfriend she has " a big pussy , " referring of course to the size of the cat hairs on her coat , but nonetheless , grant paid $60 to have the exchange included in the movie ) pass for comedy ? 
nine months is a predictable cookie-cutter movie

Ahora en la variable `raw_text` tenemos un array de caracteres o *string* (con todas las funciones de los tipos *string*). Por ejemplo, podemos ver los primeros 40 caracteres de este documento como:

In [14]:
print(type(raw_text))

print(raw_text[:40])

print('\n The total number of characters in the document is %d' %(len(raw_text)))

<class 'str'>
that's exactly how long the movie felt t

 The total number of characters in the document is 3554


# 2. Preprocesado del corpus

Antes de transformar los datos de entrada de texto en una representación vectorial, necesitamos estructurar y limpiar el texto, y conservar toda la información que permita capturar el contenido semántico del corpus.

Para ello, el procesado típico de NLP aplica los siguientes pasos:

1. Tokenización
2. Homogeneización
3. Limpieza

## 2.1. Tokenization

**Tokenización** es el proceso de dividir el texto dado en piezas más pequeñas llamadas tokens. Las palabras, los números, los signos de puntuación y otros pueden ser considerados como tokens.

Ya hemos visto que el objeto `CorpusReader` incluye funciones para dividir el corpus en frases o palabras. Pero NLTK incluye también funciones genéricas para hacer estas operaciones sobre cualquier cadena de texto. En concreto, tiene dos funciones:
- `sent_tokenize`: es un tokenizador de frases. Este tokenizador divide un texto en una lista de oraciones. Para decidir dónde empieza o acaba una frase NLTK tiene un modelo pre-entrenado para el idioma específico en el que estemos trabajando. Este modelo lo hemos cargado al principio con `nltk.download('punkt')`.
- `word_tokenize`/`wordpunct_tokenize`:  Divide un texto en palabras u otros caracteres individuales cómo pueden ser signos de puntuación.

## 2.2. Homogeneización

Al observar los tokens del corpus podemos ver que hay muchos tokens con algunas letras en mayúsculas y otras en minúsculas, el mismo token unas veces aparece en singular y otras en plural, o el mismo verbo que aparece en diferentes tiempos verbales. Para analizar semánticamente el texto, nos interesa  **homogeneizar** las palabras que formalmente son diferentes pero tienen el mismo significado.

El proceso habitual de homogeneización consiste en los siguientes pasos:

1. Eliminación de las mayúsculas y caracteres no alfanuméricos: de este modo los caracteres alfabéticos en mayúsculas se transformarán en sus correspondientes caracteres en minúsculas y  se eliminarán los caracteres no alfanuméricos, por ejemplo, los signos de puntuación. 

2. Stemming/Lematización: eliminar las terminaciones de las palabras para preservar su raíz de las palabras e ignorar la información gramatical (eliminamos marcas de plurales, género, conjugaciones verbales, ...).

3. Eliminar **stopping words** (palabras que no son en general informativas)


**Stemming and Lemmatización**

En el lenguaje común, las palabras pueden tomar diferentes formas indicando género, cantidad, tiempo (en el caso de los verbos), formas concretas para nombres/adjetivos o adverbios, ... Para muchas aplicaciones, es útil normalizar estas formas en alguna palabra canónica que facilite su análisis. Hay dos maneras de realizar este proceso:

1. El proceso de **stemming** reduce las palabras a su base o raíz 

      running --> run

      flowers --> flower

  Para poder hacer esta transformación necesitamos librerías específicas que tienen almacenadas para el vocabulario de cada idioma las raices de dicho vocabulario y hacen esta conversión. 

    
2. El objetivo de la **lematización**, al igual que el stemmer, es reducir las formas inflexionales a una forma base común. A diferencia del steming, la lematización no se limita a cortar las inflexiones. En su lugar, utiliza bases de conocimiento léxico para obtener las formas básicas correctas de las palabras. 

    women   --> woman

    foxes   --> fox
    
  

Una de las ventajas de la lematización es que el resultado sigue siendo una palabra, lo que es más aconsejable para la presentación de los resultados del procesado de textos.

**Eliminar palabras irrelevantes**

El último paso del preprocesado consiste en eliminar las palabras irrelevantes o **stop words** de los documentos. Las **stop words** son las palabras más comunes en un idioma como "el", "a", "sobre", "es", "todo". Estas palabras no tienen un significado importante y normalmente se eliminan de los textos. Para aplicar este proceso, se cargan librerías específicas que contienen este listado de palabras por cada idioma.

Además, este paso suele utilizarse para eliminar las marcas de puntaución (',', '.', '?', ....), para lo que también tenemos que cargar otro módulo con los signos de puntuación. 


## Preprocesado de texto con spaCy

[spaCy](https://spacy.io/) es una biblioteca gratuita de código abierto para el procesamiento avanzado del lenguaje natural en Python. spaCy está diseñado específicamente para uso en producción. A diferencia de NLTK, spaCy sigue una orientación de objetos. Por ejemplo, cuando tokenizamos un texto, cada token es un objeto con atributos y propiedades específicas.

Spacy da soporte para más de 64 idiomas, incluyendo modelos estadísticos ya entrenados para [17 de ellos](https://spacy.io/usage/models) (incluyendo word embeddings y modelos basados en [transformers](https://spacy.io/usage/v3), la última revolución en NLP).


Puesto que en esta sesión sólo vamos a cubrir algunos aspectos básicos de spaCy, en los siguientes recursos podéis encontrar material adicional:

- [spaCy 101 course](https://spacy.io/usage/spacy-101)
- [Advanced Tutorial](https://course.spacy.io/en/)


El procesado de texto con spaCy es sencillo. Cargaremos un modelo pre-entrenado para un determinado idioma, y pasamos cualquier texto a procesar. spaCy ejecutará una serie de procesos (pipeline) sobre el mismo y  devolverá un objeto tipo `doc`.

<figure>
<center>
<img src='https://spacy.io/images/architecture.svg' width="800"></img>
<figcaption>From Spacy documentation</figcaption></center>
</figure>

La arquitectura básica en spaCy es la siguiente:

   - `Language`: se determina al cargar el modelo y el pipeline de procesos asociados. Trasforma texto en objectos spaCy.
   - `Doc`: Secuencia iterable de tokens. 
   - `Vocab`: Diccionario asociado al modelo.  


En el siguiente código, descargamos e importamos uno de los [modelos estadísticos pre-entrenados para idioma inglés](https://spacy.io/models/en)...

In [15]:
# Para forzar spacy 3.0 (con colab se abre 2.2.X)
#!pip install --upgrade spacy

In [16]:
# Descargamos el modelo
!python -m spacy download en_core_web_sm

     |████████████████████████████████| 13.9 MB 3.7 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [17]:
import spacy

# Cargamos el modelo
nlp = spacy.load("en_core_web_sm")

El **acceso al vocabulario** en modo se hace a través del atributo `.vocab.strings`

In [18]:
lista_vocab = list(nlp.vocab.strings)

print("El tamaño del diccionario es de {} palabras".format(len(lista_vocab)))

#Las 200 primeras
print(lista_vocab[:200])

El tamaño del diccionario es de 83839 palabras
['\t', '\n', ' ', '  ', '!', '!!', '!!!', '!!!!', '!!!!!!!!!!!!!!!!', '!!!!.', '!!.', '!!?', '!!??', '!*', '!.', '!?', '!??', '"', '""', '#', "##'s", "##'x", "#'s", '#15', '#^%', '#dd', '$', '$19', '$Whose', '$Xxxxx', '$whose', '$xxxx', '%', '%-3', '%ach', '%ah', '%eh', '%er', '%ha', '%hm', '%huh', '%mm', '%oof', '%pw', '%uh', '%um', '%xx', '%xxx', '&', '&#', '&G.', '&L.', '&Ls', '&M.', '&P.', '&SA', '&T.', '&ex', '&in', '&ls', '&of', '&on', '&sa', '&the', '&to', '&uh', '&von', '&xx', '&xxx', "'", "''", "''It", "''Xx", "''it", "''xx", "'-(", "'-)", "'03", "'07", "'20s", "'30s", "'40s", "'45", "'46", "'50s", "'60s", "'67", "'68", "'69", "'70's", "'70s", "'71", "'73", "'74", "'76", "'78", "'80", "'80's", "'80s", "'82", "'86", "'89", "'90's", "'90s", "'91", "'94", "'96", "'97", "'98", "'99", "'Arabi", "'Cause", "'Connery", "'Cos", "'Coz", "'Cuz", "'Id", "'Il", "'It", "'N", "'Nita", "'S", "'T", "'X", "'Xx", "'Xxx", "'Xxxx", "'Xxxxx", "'ai", "'

Podemos ver que el vocabulario incluye ya emoticonos codificados en modo texto.

In [19]:
no_alpha_words = [word for word in nlp.vocab.strings if word.isalpha() is True]

print(f'Hay un total de {len(no_alpha_words)} palabras con caracterés alfabéticos. \n Las 20 primeras son ...')

print(no_alpha_words[:20])

Hay un total de 73384 palabras con caracterés alfabéticos. 
 Las 20 primeras son ...
['A', 'AA', 'AAA', 'AAC', 'AAV', 'AB', 'ABA', 'ABB', 'ABBA', 'ABBIE', 'ABC', 'ABCs', 'ABD', 'ABM', 'ABORTION', 'ABS', 'ABUSE', 'AC', 'ACC', 'ACCEPTANCES']


### objetos de texto en SpaCy 

A continuación, vamos a utilizar el pipeline que hemos cargado para analizar un texto ...

In [21]:
sentence = 'Japan’s economy, struggling to emerge from the pandemic, is sagging under rising food and energy costs and a weak yen'

print(sentence)

doc = nlp(sentence)

Japan’s economy, struggling to emerge from the pandemic, is sagging under rising food and energy costs and a weak yen


`doc` es un objeto iterable, compuesto por objetos tipo [`token`](https://spacy.io/api/token). En el siguiente bucle imprimimos algunas de las propiedades de dichos tokens determinadas por el pipeline que hemos cargado, incluyendo el POS tag:

In [22]:
for token in doc:
    print(token.text, token.lemma_, token.tag_, token.is_alpha, token.is_stop,token.is_punct)
    print('*****')

Japan Japan NNP True False False
*****
’s ’s POS False True False
*****
economy economy NN True False False
*****
, , , False False True
*****
struggling struggle VBG True False False
*****
to to TO True True False
*****
emerge emerge VB True False False
*****
from from IN True True False
*****
the the DT True True False
*****
pandemic pandemic NN True False False
*****
, , , False False True
*****
is be VBZ True True False
*****
sagging sag VBG True False False
*****
under under IN True True False
*****
rising rise VBG True False False
*****
food food NN True False False
*****
and and CC True True False
*****
energy energy NN True False False
*****
costs cost NNS True False False
*****
and and CC True True False
*****
a a DT True True False
*****
weak weak JJ True False False
*****
yen yen NN True False False
*****


En forma de tabla ...

In [24]:
import pandas as pd

spacy_pos_tagged = [(token.text, token.lemma_, token.tag_,token.is_alpha, token.is_stop,token.is_punct) for token in doc]

pd.DataFrame(spacy_pos_tagged, columns=['Word','Word_Lemma','POS tag','Is alpha','Is stopword','Is punct'])

,Word,Word_Lemma,POS tag,Is alpha,Is stopword,Is punct
0,Japan,Japan,NNP,True,False,False
1,’s,’s,POS,False,True,False
2,economy,economy,NN,True,False,False
3,",",",",",",False,False,True
4,struggling,struggle,VBG,True,False,False
5,to,to,TO,True,True,False
6,emerge,emerge,VB,True,False,False
7,from,from,IN,True,True,False
8,the,the,DT,True,True,False
9,pandemic,pandemic,NN,True,False,False


Con `spacy.explain()` podemos obtener una descripción de los distintos tags ...

In [25]:
spacy.explain("VBP")

'verb, non-3rd person singular present'

In [26]:
spacy.explain("JJ")

'adjective (English), other noun-modifier (Chinese)'

En lo referente a **stopping words**, spaCy incluye una amplia lista (326 elementos en idioma inglés) que podemos personalizar para nuestra propia aplicación si es necesario.

In [27]:
spacy_stopwords = nlp.Defaults.stop_words

#Printing the total number of stop words:
print('Number of stop words: %d' % len(spacy_stopwords))

#Printing first ten stop words:
print('First ten stop words: %s' % list(spacy_stopwords)[:20])

Number of stop words: 326
First ten stop words: ['once', 'about', 'ever', 'go', 'neither', 'its', 'yourselves', 'itself', 'done', 'by', 'say', 'hereafter', 'will', 'eleven', 'alone', 'hereupon', 'than', 'wherein', 'here', 'part']


In [28]:
# Añadir una palabra al conjunto de stopping words

nlp.Defaults.stop_words.add("my_new_stopword")

print('Number of stop words: %d' % len(nlp.Defaults.stop_words))

# Quitar una palabra del conjunto de stopping words

nlp.Defaults.stop_words.remove("my_new_stopword")

print('Number of stop words: %d' % len(nlp.Defaults.stop_words))



Number of stop words: 327
Number of stop words: 326


## Spacy para idioma español

Tal y como hemos comentado, Spacy proporciona modelos pre-entrenados para trabajar con idioma español. Todos han sido entrenados en la base de datos anotada [Ancora](http://clic.ub.edu/corpus/). Este corpus contiene 500.000 textos  periodísticos publicados en medios españoles. 

Es recomendable mirar la siguiente [documentación](https://spacy.io/models/es).


In [29]:
!python -m spacy download es_core_news_sm

     |████████████████████████████████| 14.0 MB 3.6 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [30]:
nlp = spacy.load("es_core_news_sm")

lista_vocab = list(nlp.vocab.strings)

print("El tamaño del diccionario es de {} palabras. Las primeras 100 son \n".format(len(lista_vocab)))

print(lista_vocab[:100])

#Printing the total number of stop words:
print('\nHay un total de {0} stopping words en el modelo'.format(len(spacy.lang.es.stop_words.STOP_WORDS)))



El tamaño del diccionario es de 181846 palabras. Las primeras 100 son 

['\t', '\n', ' ', '  ', ' el', ' tú', ' yo', ' él', '!', '"', '"""', '"Caño"', '"El', '"Guga"', '"Steel"', '"Torito"', '"Tucho"', '"Xx', '"Xxxx"', '"Xxxxx"', '"añicos', '"caño"', '"el', '"guga"', '"steel"', '"torito"', '"tucho"', '"write', '"xx', '"xxxx', '"xxxx"', '#', '$', '%', '&', "'", "' matar tú '", "' xxxx xx '", "''", "'-(", "'-)", "'13", "'23", "'70", "'92", "'96", "'Arteaga'", "'Catanha'", "'Erika'", "'Guti'", "'Or", "'Xxxx'", "'Xxxxx'", "'an", "'arteaga'", "'catanha'", "'d", "'di", "'em", "'er", "'erika'", "'ev", "'excelencia'", "'guti'", "'hooligans'", "'il", "'in", "'ir", "'ll", "'m", "'matarte'", "'mystes'", "'or", "'re", "'s", "'savoir", "'síndrome", "'ve", "'wa", "'x", "'xx", "'xxxx", "'xxxx'", "'ya", "'нs", '(', '(((', '(*>', '(*_*)', '(-8', '(-:', '(-;', '(-_-)', '(-d', '(._.)', '(:', '(;', '(=', '(>_<)', '(^_^)']

Hay un total de 551 stopping words en el modelo


### Uso de SpaCy para el preprocesamiento de texto

Como hemos visto, usando SpaCy, podemos obtener directamente a los lemas de cada token junto con algunos atributos para indicar si el token es una palabra vacía (.is_stop), alfanumérico (.is_alpha), un signo de puntuación (.is_punct) o un dígito (. es_dígito) . Podemos usar directamente esta información para preprocesar nuestro corpus.

La siguiente función implementa una canalización de normalización para un documento determinado:

In [32]:

nlp = spacy.load("en_core_web_sm")

def normalize_Spacy(text):
    text2 = nlp(text)
    normalized_text = [w.lemma_.lower() for w in text2 if not w.is_stop 
                  and not w.is_punct and (w.is_alpha or w.is_digit)]
    return normalized_text

Y ahora normalizamos todo el corpus...

In [33]:
corpus_prec = []

for fileid in movie_reviews.fileids():
    text = movie_reviews.raw(fileid)
    text_preproc = normalize_Spacy(text)
    corpus_prec.append(text_preproc)

In [34]:
len(corpus_prec)

2000

In [35]:
corpus_prec[0]

['plot',
 'teen',
 'couple',
 'church',
 'party',
 'drink',
 'drive',
 'accident',
 'guy',
 'die',
 'girlfriend',
 'continue',
 'life',
 'nightmare',
 'deal',
 'watch',
 'movie',
 'sorta',
 'find',
 'critique',
 'mind',
 'fuck',
 'movie',
 'teen',
 'generation',
 'touch',
 'cool',
 'idea',
 'present',
 'bad',
 'package',
 'make',
 'review',
 'hard',
 'write',
 'generally',
 'applaud',
 'film',
 'attempt',
 'break',
 'mold',
 'mess',
 'head',
 'lose',
 'highway',
 'memento',
 'good',
 'bad',
 'way',
 'make',
 'type',
 'film',
 'folk',
 'snag',
 'correctly',
 'take',
 'pretty',
 'neat',
 'concept',
 'execute',
 'terribly',
 'problem',
 'movie',
 'main',
 'problem',
 'simply',
 'jumbled',
 'start',
 'normal',
 'downshift',
 'fantasy',
 'world',
 'audience',
 'member',
 'idea',
 'go',
 'dream',
 'character',
 'come',
 'dead',
 'look',
 'like',
 'dead',
 'strange',
 'apparition',
 'disappearance',
 'looooot',
 'chase',
 'scene',
 'ton',
 'weird',
 'thing',
 'happen',
 'simply',
 'explain',


## Contando las palabras más frecuentes en nuestro corpus

Utilizando la clase `Counter` dentro de la librería `collections` podemos encontrar facilmente las palabras más utilizadas en nuestro corpus ya preprocesado:

In [39]:
from collections import Counter

words_freq = Counter([w for d in corpus_prec for w in d])

In [41]:
common_words = words_freq.most_common(20)

In [42]:
print(common_words)

[('film', 11171), ('movie', 6972), ('like', 3951), ('character', 3874), ('good', 3590), ('time', 2984), ('scene', 2670), ('story', 2343), ('play', 2324), ('come', 1969), ('know', 1965), ('bad', 1940), ('man', 1932), ('go', 1911), ('way', 1879), ('look', 1875), ('life', 1797), ('year', 1728), ('work', 1706), ('thing', 1661)]


> **Ejercicio**: Calcule los adjetivos más frecuentes en el corpus